<img align="center" width="12%" style="padding-right:10px;" src="./Images/Ccom.png">

# Integrated Seabed Mapping Systems <a href="https://teams.microsoft.com/l/channel/19%3a6d420007fb504c57850639e54a52945b%40thread.tacv2/Lab%2520D?groupId=b7209537-725b-40fc-bdbe-212e9689ef18&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="./Images/help.png"  title="Ask questions on Teams" align="right" width="10%" ></a><br><br>  Lab D: 

___
## D0 Create Python Script - Load the Dependecies

We will run this lab a little different. Rather than adding and editing code in this Notebook we will use a script that we will run. This has the advantage that we ensure that all the code is executed sequentially. It also highlights another capability of Jupyter Notebooks.

Create a LabD.py File and load the dependencies 

    # D0 Create Python Script - Load the Dependecies

    import sys
    import import_ipynb
    import os.path
    import matplotlib as plt
    import numpy as np
    from datetime import datetime, timedelta
    from numpy import pi, arctan2, arccos, abs, sin, cos, tan, sqrt, sum, arctan
    from mycode.position import Position
    from mycode.twtt import TWTT
    from mycode.integration import Integration
    from mycode.integration import Motion
    from datetime import datetime, timezone
    from mycode.vessel import Vessel
    from mycode.ssp import SSP
    from mycode.ping import Ping
    from mycode.om_math import Rx, Ry, Rz



___
## D0.1 Add the Current Folder To the Path

Before we start in earnest we will have to do some more house keeping, we need to add the current folder to the path

    # D0.1 Add the Current Folder To the Path

    sys.path.append(os.getcwd())
    abs_path=os.path.abspath(os.path.curdir)
    print("Configured environment for Lab D")



## D0.2 Testing, Testing, D0.1, D0.2

In the code cell below we should see whether things worked



In [59]:
%load_ext autoreload
%autoreload 2

%run -i LabD.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]


___
## D1 Defining the Vessel

<img align="center" width="80%" style="padding-right:10px;" src="./Images/conecone.jpg">

In Lab A we saw that to obtain a location and orientation of the sonars (from which to calculate their sonar-relative beam vectors) we need to utilize asynchronous observations of position and orientation. They are both available on the vessel, but at different locations or orientations. To relate the two together, we needed to establish a ship reference frame.

Within that reference frame we needed to establish the location and orientation of each sensor in turn. The net result is that we know the location of the transmit-transducer at the transmit epochs and the receive-transducer at the receive epochs.

The data presented here is from the same expedition as that presented in Lab A, i.e. the vessel data is the same
In the `LabD.py` script add the following to define the a `Vessel` object name `vessel`

    vessel = Vessel()
    vessel.metadata["name"]="USNS Henson"
    vessel.metadata["owned_by"]="United States Navy"
    vessel.metadata["operated_by"]="United States Navy"
    vessel.metadata["pos_source"]="NavCom (C-Nav)"
    vessel.metadata["sonar"]="Kongsberg EM710"
    vessel.metadata["mru"]="Applanix POS/MV 320"
    vessel.metadata["loa"]=100
    
Also add the lever arms and water line

    vessel.wl = -2.59
    vessel.lever_arm_trans = np.asarray([16.26, -1.75, 4.15]).reshape((3, 1))
    vessel.lever_arm_rec = np.array([14.82, -2.01, 4.17]).reshape((3, 1))
    vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))
    vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))


___
## D11.0 Georeferencing the Antenna Lever Arms

First we will calculate the georeferenced GNSS lever arm `geo_tx_ant` for epoch `t_tx` and geo_tx_ant` for epoch `t_rx`

In the LabD.py script add and complete the following:

    geo_tx_ant = positions.get_position(...)
    geo_rx_ant = ...


In [60]:
%run -i LabD.py

print(vessel)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
Vessel Metadata: name: USNS Henson
owned_by: United States Navy
operated_by: United States Navy
loa: 100
pos_source: NavCom (C-Nav)
sonar: Kongsberg EM710
mru: Applanix POS/MV 320
dist_unit: m
angle_unit: rad



___
## D2 Representing the Transmit and Receive Array

For the directions it is helpful to represent the transducer arrays by unit vectors. Unit vectors have the unique property that the elements are the direction cosines with respect to the axis. We could proof this formally, but 
we will illustrate this at the hand of an example: the vector (1,1) has an angle of $1/4 \pi$ radians and a length of $\sqrt{2}$ Thus if we normalize the vector we end up with a vector $\left(\sqrt{2},\sqrt{2}\right)$ which has norm $1$. Similarly the $\cos(1/4 \pi) = \sqrt{2}$. 

Thus it is very convenient to represent direction using unit vectors, the elements are direct measures of the angle that the vector makes with the coordinate axes. i.e. the first element represents the cosine of the angle of the vector with the first coordinate axis, the 2nd with the 2nd axis, etc.

We will chose some directions of convenience - ideally the transmit transducer is aligned to the X-axis of a vessel thus a logical vector to represent its orientation is (1,0,0), ( the angle with the x-axis is zero, thus the first element is cos(0) = 1, the angle with the y- and z-axes is 90 degrees cos(90) = 0 thus our vector is (1, 0, 0)

We will represent the ideal transmit and receive transducers with the variable `tx_ideal` and `rx_ideal` respectively

In the `LabD.py` script add and complete the following 

    tx_ideal = np.array([[1],[0],[0]])
    rx_ideal = np.array([...])


In [61]:
%run -i LabD.py

print( tx_ideal)
print( rx_ideal)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 1.0000]
 [ 0.0000]
 [ 0.0000]]
[[ 0.0000]
 [ 1.0000]
 [ 0.0000]]


    Configered environment for Lab D
    [[1]
     [0]
     [0]]
    [[0]
     [1]
     [0]]


___
## D2.0 Representing the Transmit and Receive Array - Reality is Not Ideal

Unfortunately reality is not ideal. The transducer will not be mounted quite orthogonal and not quite aligned to the vessel reference frame. We will have to deal with some mount angles that are deviations from the ideal vectors. In the case of the 'Henderson' these are known, so we may take them into account.

In the `LabD.py` script add the following: 
    ma_tx=np.array([0.127,1.024,359.957-360])*pi/180  # Tx Mount angles 
    ma_rx=np.array([0.101,0.894,0.065])*pi/180    # Rx Mount angles 



In [62]:
%run -i LabD.py

print( ma_tx)
print( ma_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[ 0.0022  0.0179  -0.0008]
[ 0.0018  0.0156  0.0011]


    Configered environment for Lab D
    [ 0.00221657  0.01787217 -0.00075049]
    [0.00176278 0.01560324 0.00113446]

___
## D3 Parameters for Beam Forming

In our case we will be reading data from a datagram collected with the Kongsberg system - all the data associated to the specific beam for the specific ping that we are interested in is contained in the data file `data_ping_4170.txt` that you may find in your `Data` folder.

In our particular case you will be analyzing beam 391, we will keep track of that here so that you may later on extract the right information for the beam depression angle. We will also give you the sound speed at the transducer here, as you should not use the sound speed from the sound speed profile for that - that would create a significant error in your ray tracing.

In the `LabD.py` script add the following:
    ss_tx=1543.7                       
    beam_select=391   

In [63]:
%run -i LabD.py

print( ss_tx)
print( beam_select)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
1543.7
391


    Configered environment for Lab D
    1543.7
    391


___
## D4 Get the Positioning Data

We will get the positioning data and store them in a `Position` object called `positions`. We will do this in the same fashion as in Lab A - the data file is called `Lab_A_GNSS.txt` is located in the Data folder, should be read with the `read_jhc_file` method:

In the `LabD.py` script add and complete the following:
    positions = Position()
    positions.read_jhc_file(abs_path+...)

In [64]:
%run -i LabD.py

print( positions)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
geodetic_units: rad
height_units: m
proj_units: m
geoid_name: EGM08
ellipsoid_name: WGS84
height_relative_to: geoid
time_basis: UTC
proj_str: +proj=utm +zone=55 +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs
Minimum latitude       : 0.278496
Maximum latitude       : 0.278564
Minimum longitude      : 2.548137
Maximum longitude      : 2.548780
Minimum height         : 27.18m
Maximum height         : 82.38m
Start Time             : 2011-05-08 03:49:11+00:00
End Time               : 2011-05-08 03:49:11+00:00



    Configered environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    geodetic_units: rad
    height_units: m
    proj_units: m
    geoid_name: EGM08
    ellipsoid_name: WGS84
    height_relative_to: geoid
    time_basis: UTC
    proj_str: None
    Minimum latitude       : 0.278496
    Maximum latitude       : 0.278564
    Minimum longitude      : 2.548137
    Maximum longitude      : 2.548780
    Minimum height         : 27.18m
    Maximum height         : 82.38m
    Start Time             : 2011-05-08 03:49:11+00:00
    End Time               : 2011-05-08 03:49:11+00:00



___
## D4.0 Squaring the Positioning Data Away

As we saw in lab A we want to do our calculations in a Cartesian world if we can. Thus, just like in that lab, we want to transform our coordinates to UTM coordinates. For this we will use the `Position` class `carto_project` method

In the `LabD.py` script add the following:

    positions.carto_project("UTM", "ortho")

In [65]:
%run -i LabD.py

print( positions.proj_pos)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 392719.3368  392724.8274  392730.7915 ...  392723.9758  392718.4610
   392713.0224]
 [ 1764811.7544  1764811.9935  1764812.2945 ...  1764405.1255
   1764405.2317  1764405.3961]
 [ 28.8950  29.1460  28.9800 ...  28.8460  28.7760  82.3750]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    [[3.92719337e+05 3.92724827e+05 3.92730791e+05 ... 3.92723976e+05
      3.92718461e+05 3.92713022e+05]
     [1.76481175e+06 1.76481199e+06 1.76481229e+06 ... 1.76440513e+06
      1.76440523e+06 1.76440540e+06]
     [2.88950000e+01 2.91460000e+01 2.89800000e+01 ... 2.88460000e+01
      2.87760000e+01 8.23750000e+01]]


___
## D5 Get the Motion Data

We will get the motion data and store them in a `Motion` object called `motions`. Just like the positioning we will do this in the same fashion as in Lab A - the data file is called `Lab_A_MRU.txt` is located in the Data folder and should be read with the `read_jhc_file` method:

In the `LabD.py` script add and complete the following:

    positions = Position()
    positions.read_jhc_file(...)

In [66]:
%run -i LabD.py

print(motions)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
Start time: 2011-05-08 03:49:10.572000+00:00
End time:   2011-05-08 04:12:27.053000+00:00
angle__units: rad
distance_units: m
time_basis: UTC
Source File: /home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt



    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Start time: 2011-05-08 03:49:10.572000+00:00
    End time:   2011-05-08 04:12:27.053000+00:00
    angle__units: rad
    distance_units: m
    time_basis: UTC
    Source File: /home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt



___
## D6 Get the Sound Speed Data

We will get the sound speed data and store them in a `SSP` object called `ssp`. This time the data file is called `Lab_A_SVP.txt` and is located in the Data folder. You should read it with the `read_jhc_file` method which you may add to the SSP class.

    def read_jhc_file(self, fullpath):
        # Check to see whether data already exists in the object
        
        if self.obs_depths:
            raise RuntimeError('SSP object already contains a profile')
            
        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening sound speed profile data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        svp_file = open(fullpath)
        svp_content = svp_file.read()
        svp_file.close

        # Tokenize the contents
        svp_lines = svp_content.splitlines()
        self.obs_time = datetime.fromtimestamp(
            float(svp_lines[1].split()[0]), timezone.utc)
        self.log_time = datetime.fromtimestamp(
            float(svp_lines[2].split()[0]), timezone.utc)
        self.obs_latitude = float(svp_lines[3].split()[0])
        self.obs_longitude = float(svp_lines[3].split()[1])
        self.vessel_latitude = float(svp_lines[4].split()[0])
        self.vessel_longitude = float(svp_lines[4].split()[1])
        self.metadata["count"] = int(svp_lines[5].split()[0])

        count = 0  # initialize the counter for the number of rows read

        for svp_line in svp_lines[16:]:
            observations = svp_line.split()  # Tokenize the stringS
            self.obs_sample.append(float(observations[0]))
            self.obs_depths.append(float(observations[1]))
            self.obs_ss.append(float(observations[2]))
            count += 1

        if self.metadata["count"] != count:
            raise RuntimeError('Nr of Samples read ('+str(count) +
                               ') does not match metadata count (' +
                               str(self.metadata["count"])+')')

        # Process the data - in the jhc data files this is already a one-way profile,
        # this just for illustration
        array_ss = np.zeros((count, 3))

        # Sort the data samples by depth
        sorted_ss = sorted(zip(self.obs_depths, self.obs_ss))
        
        layer = 0
        for d, ss in sorted_ss:
            array_ss[[layer], [0]] = d
            array_ss[[layer], [1]] = ss
            layer += 1

        # Identify all the depths for which there are multiple observations
        mask = np.full((count, 1), True)
        mask[1:, [0]] = np.diff(array_ss[:, [0]], axis=0) != 0
        
        # Remove the duplicates - You really should get statistical representations here
        # but to keep this short just remove the duplicates
        array_ss = array_ss[mask[:, 0], ...]
        
        # Determine the gradients - Note the indexing: the gradient of the first layer 
        # is contained at the same index as the data for the TOP of the layer.
        array_ss[0:-1, [2]] = np.diff(array_ss[:, [1]],
                                          axis=0)/np.diff(array_ss[:, [0]], axis=0)

        # Estimate gradient for last layer assuming that the temperature and salinity remain the same
        # gradient solely a function of pressure (depth)
        array_ss[-1, [2]] = 0.017

        # Extend to 12000 m if necesarry - this is to get around some manufcturers requirements
        if self.obs_depths[-1] < 12000:
            ss = array_ss[-1:, [1]] + array_ss[-1:, [2]] \
             * (12000-array_ss[-1:, [0]])
            array_ss = np.vstack((array_ss, [12000, ss, 0.017]))
            
        # Make sure that the last gradient is 0.017
        array_ss[-1,2] = 0.017

        # Extend to 0 m if necesarry - assume well mixed
        if self.obs_depths[0] > 0:
            array_ss = np.vstack(
                ([0, array_ss[0, [1]], 0.], array_ss))
            
        # Step 5 Create a look-up array of twtts for each full layer
        # Allows for great gain in efficiency (do not have to calculate for each ping)
        self.twtt_layer = np.zeros((count, 1))
        
        for layer in range(0,self.metadata["count"]-1):
            if array_ss[layer, [2]] == 0:
                self.twtt_layer[layer] = 2 * \
                    (array_ss[layer+1, [0]] - array_ss[layer, [0]])/ \
                     array_ss[layer, [1]]
            else:
                self.twtt_layer[layer] = 2 / array_ss[layer, [2]] * \
                 log(array_ss[layer+1, [1]]/array_ss[layer, [1]])
        
        self.proc_ss = array_ss[:,1]
        self.proc_depth = array_ss[:,0]
        self.g = np.diff(self.proc_ss)/np.diff(self.proc_depth)
        self.g[self.g == 0] = .24
        
        # Updating the Profile
        for i in range(1,len(self.g)):
            self.proc_ss[i]=self.proc_ss[i-1]+(self.proc_depth[i] - self.proc_depth[i-1])*self.g[i-1]


In the `LabD.py` script add and complete the following:

    ssp = ...
    ssp.read_jhc_file(...)

In [67]:
%run -i LabD.py

print(ssp.proc_depth)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[ 0.0000  2.0000  3.0000  4.0000  5.0000  6.0000  7.0000  8.0000  9.0000
  10.0000  11.0000  12.0000  13.0000  14.0000  15.0000  16.0000  17.0000
  18.0000  19.0000  20.0000  21.0000  22.0000  23.0000  24.0000  25.0000
  26.0000  27.0000  28.0000  29.0000  30.0000  31.0000  32.0000  33.0000
  34.0000  35.0000  36.0000  37.0000  38.0000  39.0000  40.0000  41.0000
  42.0000  43.0000  44.0000  45.0000  46.0000  47.0000  48.0000  49.0000
  50.0000  51.0000  52.0000  53.0000  54.0000  55.0000  56.0000  57.0000
  58.0000  59.0000  60.0000  61.0000  62.0000  63.0000  6

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    [0.0e+00 2.0e+00 3.0e+00 4.0e+00 5.0e+00 6.0e+00 7.0e+00 8.0e+00 9.0e+00
     1.0e+01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01 1.7e+01 1.8e+01
     1.9e+01 2.0e+01 2.1e+01 2.2e+01 2.3e+01 2.4e+01 2.5e+01 2.6e+01 2.7e+01
     2.8e+01 2.9e+01 3.0e+01 3.1e+01 3.2e+01 3.3e+01 3.4e+01 3.5e+01 3.6e+01
     3.7e+01 3.8e+01 3.9e+01 4.0e+01 4.1e+01 4.2e+01 4.3e+01 4.4e+01 4.5e+01
     4.6e+01 4.7e+01 4.8e+01 4.9e+01 5.0e+01 5.1e+01 5.2e+01 5.3e+01 5.4e+01
     5.5e+01 5.6e+01 5.7e+01 5.8e+01 5.9e+01 6.0e+01 6.1e+01 6.2e+01 6.3e+01
     6.4e+01 6.5e+01 6.6e+01 6.7e+01 6.8e+01 6.9e+01 7.0e+01 7.1e+01 7.2e+01
     7.3e+01 7.4e+01 7.5e+01 7.6e+01 7.7e+01 7.8e+01 7.9e+01 8.0e+01 8.1e+01
     8.2e+01 8.3e+01 8.4e+01 8.5e+01 8.6e+01 8.7e+01 8.8e+01 8.9e+01 9.0e+01
     9.1e+01 9.2e+01 9.3e+01 9.4e+01 9.5e+01 9.6e+01 9.7e+01 9.8e+01 9.9e+01
     1.2e+04]

___
## D7 Get the Ping Data

<img align="center" width="80%" style="padding-right:10px;" src="./Images/ping.png">

A single file *data_ping_4170.txt* is supplied which, together with the orientation and offsets will allow you to do the full calculation.

What information do you have on the transmit sector for the beam you chose?
From the data telegram, there is lots of other information that is provided about each sector. The only things you need are:

- the transmit steering angle and
- the time delay between the first sector firing and the one you have chosen.

Your actual time of transmission is the nominal time of the swath PLUS the extra delay while it waits for the other sectors to fire. For example, Sector 1 fired first (at the specified time). Sector 0 fires 0.4ms later and Sector 2 fire another 0.4ms after that (0.8ms w.r.t the central sector).

To save you some time a `Ping` class has been created for you - you may find it in your `mycode` folder and it is already included in the dependencies.


In the `LabD.py` script add and complete the following:

    ssp = ...
    ssp.read_jhc_file(...)

In [68]:
%run -i LabD.py

print(ping)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
Ping     : 4170
Latitude : 15.957991°
Longitude: 146.03402800000003°
Roll     : -1.054242°
Pitch    : 2.0454589999999997°
Headding : 215.529534°
Heave    : -0.542 [m]
Tx Time  : 04:01:06  May 8, 2011

3 Sectors --- 
--- Sector 0
Tx Tilt Angle   : -1.91°
Focus Range     : 146.7 [m]
Signal Length   : 0.0003 [s]
Tx time offset  : 0.0004 [s]
Tx central freq : 73000.0 [Hz]
Mean att coeff  : 20.36 [dB/km]
Waveform ID     : 0
Tx bandwidth    : 5000.0 [Hz]
--- Sector 1
Tx Tilt Angle   : -2.91°
Focus Range     : 101.1 [m]
Signal Length   : 0.0003 [s]
Tx time offset  : 0.

___
## D7.0 Indexing the Ping Data

As you can see from the output above the data comes with a header that describes the properties of three sectors. In the figure above you can see that this sector data is followed by a table of beam specific data. One thing to realize is that the beam number is associated to a specific beam, not the line number of the data in the table. To clarifu it may be that beams 1,2,4 and, 5 were processed successfully, but beam 3 failed. The data for beam 3 is then not included in the table. This has a result that the data for beams 1 and 2 are in records 1 and 2, but the data for beams 4 and 5 are in records 3 and 4 respectively. 

Thus we need a mechanism to identify the proper record, fortunately the `Ping` class comes with the `get_beam_index` method that allows us to find the record index associated to our selected beam `beam_select`

In the LabD.py script add the following:

    i = ping.get_beam_index(beam_select)


In [69]:
%run -i LabD.py

print(i)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
391


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    391


To use analyze beam data we may now use the index that we found, which in this case is the same as the beam number itself. For example, we may interested in what the sector time delay is for the beam of interest, we may find this using: 

    ping.tx_t_offset_beam[i]
    
Which will result in a `timedelta` 

In [70]:
print(ping.tx_t_offset_beam[i])

0:00:00.000800


    0:00:00.000800

___
## D8 Timing it Right

<img align="center" width="80%" style="padding-right:10px;" src="./Images/timing.bmp">

As you saw in Lab A all the integration is done based on time. We will need to know how the transducer are oriented at both the time of transmit and reception. Thus the first thing that we need to so is to determine what those points in time are. The ping time is contained in the variable `ping.tx_time`

In the `LabD.py` script add the following:

    t_tx = ping.tx_time+ping.tx_t_offset_beam[i]
    t_rx = t_tx+timedelta(0,ping.twtt[i])

What the code above does is to determine the time `t_tx` associated to the transmit, `ping.tx_time` is a `datetime` object and `ping.tx_t_offset_beam[i]` is a `timedelta` object, which allows us to handle the time more easily (I strongly suggest that you look at the `Ping` class implementation to gather greater understanding.

The next line then adds the TWTT for the beam to the transmit time to determine the reception time `t_rx`. However, `ping.twtt[i]` is a simple float and thus needs to be transformed to a `timedelta` before we may add it to `t_tx`.





In [71]:
%run -i LabD.py

print(t_tx)
print(t_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
2011-05-08 04:01:06.555800+00:00
2011-05-08 04:01:06.785567+00:00


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    2011-05-08 04:01:06.555800+00:00
    2011-05-08 04:01:06.785567+00:00

___
## D9 Getting the Motion Data for our Epochs

Now that we have the time we can orient the transducers at the time of transmit and reception. Unlike in lab A we only need to do this for a single set of transmit and receive epochs, however the same methods may be used. In our case we will add a few methods to the `Motion` class that will allow us to gather the right information



___
## D9.0 Getting Motion Vectors

The first thing that we are interested in is the orientation of the vessel at specific epochs - we will add the method
`get_motion` to the `Motion` class. This method will take a `datetime` argument an return the motion as a one dimensional `numpy` array. The order will be the rotation around the x-axis, y-axis, z-axis and the heave.

The implementation is really the same process that we used in the `Integration` class of Lab A.


**NOTE: IF YOU TOOK ESCI 872 YOU WILL ALREADY HAVE THIS METHOD**

    def get_motion(self, time = datetime.fromtimestamp(0, timezone.utc)):

        # Allocate Memory
        attitude = np.zeros(4)
        
        # 7.10.2 Map the Input Times to POSIX Times
        times = np.array([e.timestamp() for e in self.times])
        
        attitude[0] = np.interp(time.timestamp(), times, self.roll)
        attitude[1] = np.interp(time.timestamp(), times, self.pitch)
        attitude[2] = np.interp(time.timestamp(), times, self.yaw)
        attitude[3] = np.interp(time.timestamp(), times, self.heave)


In the LabD.py script add and complete the following:

    att_tx = motions.get_motion(t_tx)
    att_rx = motions.get_motion(...)

In [72]:
%run -i LabD.py

print(att_tx)
print(att_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[ -0.0188  0.0365  3.7564  0.0378]
[ -0.0277  0.0356  3.7634  0.1400]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [ -0.0188  0.0365  3.7564  0.0378]
    [ -0.0277  0.0356  3.7634  0.1400]


___
## D9.1 Getting Orientation Matrices

For many of our purposes it makes more sense to use a rotation matrix to represent our orientation and leave out the heave, which is a translational motion. For this you should add the following method to your `Motion` class

    def get_rotation_matrix(self, time = datetime.fromtimestamp(0, timezone.utc)):
        # 7.11.1 Determining the Attitude
        att = self.get_motion(time)

        Rx = np.array([[1, 0,            0          ],
                       [0, cos(att[0]), -sin(att[0])],
                       [0, sin(att[0]),  cos(att[0])]])

        Ry = np.array([[ cos(att[1]),  0, sin(att[1])],
                       [ 0          ,  1, 0          ],
                       [-sin(att[1]),  0, cos(att[1])]])

        Rz = np.array([[cos(att[2]), -sin(att[2]), 0],
                       [sin(att[2]),  cos(att[2]), 0],
                       [0          ,  0          , 1]])


In the LabD.py script add and complete the following:

    R_tx = motions.get_rot_matrix(t_tx)
    R_rx = motions....()

In [73]:
%run -i LabD.py

print(R_tx)
print(R_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ -0.8163  0.5773  -0.0190]
 [ -0.5764  -0.8163  -0.0364]
 [ -0.0365  -0.0188  0.9992]]
[[ -0.8123  0.5831  -0.0128]
 [ -0.5821  -0.8120  -0.0432]
 [ -0.0356  -0.0276  0.9990]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ -0.8163  0.5773  -0.0190]
     [ -0.5764  -0.8163  -0.0364]
     [ -0.0365  -0.0188  0.9992]]
    [[ -0.8123  0.5831  -0.0128]
     [ -0.5821  -0.8120  -0.0432]
     [ -0.0356  -0.0276  0.9990]]


___
## D10 Finding out Where We Are 

Next up is the determination of the vessel location at our epochs of interest - we will add the method
`get_position` to the `Position` class. This method will take a `datetime` argument an return the position as a one dimensional `numpy` array. The order of the elements should be the same as that in the `Position.proj_pos` array.

    def get_position(self, time = datetime.fromtimestamp(0, timezone.utc)):
        pos = np.zeros(3)
        times = np.array([e.timestamp() for e in self.times])
        pos[0] = np.interp(time.timestamp(), times, self.proj_pos[0])
        pos[1] = ...
        pos[2] = ...
        
        return pos
        
In the LabD.py script add and complete the following:

    geo_tx_ant = positions.get_position(t_tx)
    geo_rx_ant = positions.get_position(t_rx)
    
This gives the georeferenced position of the GNSS antenna at the transmit epoch `geo_tx_ant` and receive epoch `geo_rx_ant`.  

In [74]:
%run -i LabD.py

print(geo_tx_ant)
print(geo_rx_ant)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[ 396628.0730  1764532.9539  28.5175]
[ 396627.7979  1764531.9595  28.4045]


___
## D11 Georeferencing the Lever Arms

Just like in Lab A we will need to determine our lever arms in georeferenced space. We will need to geo-reference the antenna lever arm to be able to position the reference point (rp) at both the transmit and receive epochs. Similarly we will need the georeferenced transducer lever arms to be able to locate the transducer using the rp in geo-referenced space. We may use the `Motion.geo_reference_la` method for this purpose.

First we will calculate the georeferenced GNSS lever arm `geo_tx_ant` for epoch `t_tx` and then `geo_tx_ant` for epoch `t_rx`

In the LabD.py script add and complete the following:

    geo_pos_la_tx = motions.geo_reference_la(t_tx, vessel.lever_arm_pos)
    geo_pos_la_rx = ...




In [75]:
%run -i LabD.py

print(geo_pos_la_tx)
print(geo_pos_la_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 5.1770]
 [ 4.4920]
 [ -29.7635]]
[[ 4.9700]
 [ 4.7300]
 [ -29.7619]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ 5.1770]
     [ 4.4920]
     [ -29.7635]]
    [[ 4.9700]
     [ 4.7300]
     [ -29.7619]]


___
## D12 Calculating the Reference Position RP

Again like lab A we need to determine the location of our reference point `geo_pos_rp_tx` at `t_tx` and `geo_pos_rp_rx` at `t_rx`. We will store these locations in 3x1 column vectors. Just like in Lab A you have to be careful to be careful about axis alignment:

In the LabD.py script add and complete the following:

    geo_pos_rp_tx = np.zeros((3,1))
    geo_pos_rp_rx = ...

    geo_pos_rp_tx[0] =  geo_tx_ant[0] - geo_pos_la_tx[1]
    geo_pos_rp_tx[1] =  geo_tx_ant[1] - geo_pos_la_tx[0]
    geo_pos_rp_tx[2] = -geo_tx_ant[2] - geo_pos_la_tx[2]

    geo_pos_rp_rx[0] =  ...
    geo_pos_rp_rx[1] =  ...
    geo_pos_rp_rx[2] =  ...


In [76]:
%run -i LabD.py

print(geo_pos_rp_tx)
print(geo_pos_rp_rx)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 396623.5810]
 [ 1764527.7769]
 [ 1.2460]]
[[ 396623.0679]
 [ 1764526.9894]
 [ 1.3574]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ 396623.5810]
     [ 1764527.7769]
     [ 1.2460]]
    [[ 396623.0679]
     [ 1764526.9894]
     [ 1.3574]]


----
## D13 The Separation of Shot and Receive

<img align="center" width="80%" style="padding-right:10px;" src="./Images/separation_shot_receive.jpg">

As there is a finite time between transmission and reception, both the position and the orientation of the array will change in that interval. That interval can typically range from ~1/10 of second (<75m ranges) to 10’s of seconds for deep water systems.

If a single location value is to be used, the average of the transmitter position at transmit and receiver position at the average received TWTT can reasonably be used, as we did in Lab A. Strictly though, over the reception cycle, the receiver does propagate forward. But at typical vessel speeds (~4m/s) the distance traveled, compared to the one way sound speed (750 m/s) is a very small fraction (0.5%) which is smaller than the beam footprint. For example, the projected beam width of a 0.5$^{\circ}$ beam is 0.87% of Z (strictly slant range). Both these numbers are small compared to IHO Special Order horizontal accuracy requirements of 2m in 40m of water which corresponds to only 5% of Z.

**Note** that this excludes the physical separation of the transmitter and receiver acoustic centers (normally at least ½ the physical array length). If bottom detection is attempted in the acoustic near field, that separation may be significantly larger than the forward propagation of the sonar over the ping cycle.

Unlike lab A we cannot take a simple average, the beam steering is applied at the specific transmit and receive times, so we will create a virtual transducer with the transmit element located and oriented at `t_tx` and the receive element located and oriented at `t_rx`. Thus we need to have a vector that describes the distance traveled between those two epochs. We will use the vector `geo_pos_diff` for that purpose.

In the LabD.py script add the following:

    geo_pos_diff = geo_pos_rp_rx - geo_pos_rp_tx

In [77]:
%run -i LabD.py

print(geo_pos_diff)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ -0.7875]
 [ -0.5131]
 [ 0.1114]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ -0.5131]
     [ -0.7875]
     [ 0.1114]]


----
## D14 Course Over Ground

Now that we have this vector we know what the course over ground (cog) between the transmit and receive epoch was.

Note that after this it is more convenient to change the order in the `geo_pos_diff` vector

In the LabD.py script add the following:

    cog = arctan2(geo_pos_diff[0],geo_pos_diff[1])
    
    temp = geo_pos_diff.copy()
    geo_pos_diff[0] = temp[1]
    geo_pos_diff[1] = temp[0]

In [78]:
%run -i LabD.py

print(cog)
print(geo_pos_diff)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[ -2.5641]
[[ -0.7875]
 [ -0.5131]
 [ 0.1114]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [ -2.5641]
    [[ -0.5131]
     [ -0.7875]
     [ 0.1114]]


----
## D15 Drift Angle

The difference between the cog and the heading is known as the drift angle

Pet peeve alert (ignore if you like): often the drift angle is incorrectly referred to as the crab angle. The crab angle is actually a course correction applied by the navigator to achieve the desired course over ground! the Crab angle should therefore be in the opposite direction of the drift angle, but may be of a different magnitude.

remember that the heading at transmit is given by `att_tx[2]`

In the LabD.py script add and complete the following:

    da=(...-...[2])[0]
    while da < -pi:
         da = da + 2 * pi
    while da > pi:
        d...

In [79]:
%run -i LabD.py

print(da)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
-0.037315440834639446


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    -0.037315440834639446

----
## D16 Heading Change

The next thing we need is to determine the heading change between the epochs, as this will contribute to the difference in alignment between the transmit transducer at `t_tx` and the receive transducer at `t_rx`.

In the LabD.py script add and complete the following:

    d_h=att_rx[2]-att_tx[2]

In [80]:
%run -i LabD.py

print(d_h)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]
0.006934786545932159


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    0.006934786545932159


---
## Equivalent Modeled Virtual Array

<img align="center" width="80%" style="padding-right:10px;" src="./Images/eq_model.jpg">

To calculate a single beam vector for the outgoing and incoming ray path, an equivalent array geometry needs to be defined in which the two acoustic centers are co-located.

The figure above illustrates the modeled beam vector solution. A virtual array location, half way in X,Y and Z from the acoustic center at transmit and receive is calculated. The arrays, however, are no longer mutually orthogonal (even if the physically constructed arrays actually are). This is because time has elapsed.

The virtual concentric array model, adopts the transmit orientation for the transmitter and the receive orientations for the receiver.


----
## D17 Creating A new Reference System

You can now calculate the orientation of the long axis of the transmitter and receiver at the transmit and receive epochs. Recall from class that the steered beams are oriented around those long axes.

For our calculations we will assume that the transducer is aligned to the X and Y-axes, which had we not moved between the transmit and receive epochs would be simple. However we have all the information needed to construct the virtual array. We know the orientation of the tx array at transmit and the rx array at reception. We also know how the vessel moved in the intervening epoch. If we assume that the transmit transducer is aligned to the x-axis  then we need to determine where the receive transducer is relative to the transmit transducer. We do have the vector `ref_pos_diff` which tells us how far and in what direction the vessel traveled (along the ground). Thus if we want to base a new reference frame based on the orientation of the transducer we need to rotate the difference vector of the rp locations by the negative amount of the heading at transmit.

In the LabD.py script add and complete the following:

    ref_pos_diff = Rz(-att_tx[2])@geo_pos_diff



In [85]:
%run -i LabD.py

print(ref_pos_diff)

Configured environment for Lab D
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
[[ 0.9393]
 [ -0.0351]
 [ 0.1114]]


    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ 0.9393]
     [ -0.0351]
     [ 0.1114]]
